In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import os, sys, cv2
import glob
import shutil rpn_rois

sys.path.append(os.getcwd())
from lib.networks.factory import get_network
from lib.fast_rcnn.config import cfg, cfg_from_file
from lib.fast_rcnn.test import test_ctpn
from lib.utils.timer import Timer
from lib.text_connector.detectors import TextDetector
from lib.text_connector.text_connect_cfg import Config as TextLineCfg


/Users/vladimir/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'shutilrpn_rois'

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline

In [ ]:
def resize_im(im, scale, max_scale=None):
    f = float(scale) / min(im.shape[0], im.shape[1])
    if max_scale != None and f * max(im.shape[0], im.shape[1]) > max_scale:
        f = float(max_scale) / max(im.shape[0], im.shape[1])
    return cv2.resize(im, None, None, fx=f, fy=f, interpolation=cv2.INTER_LINEAR), f

In [16]:
def draw_boxes(img, image_name, boxes, scale):
    base_name = image_name.split('/')[-1]
    with open('data/results/' + 'res_{}.txt'.format(base_name.split('.')[0]), 'w') as f:
        for box in boxes:
            if np.linalg.norm(box[0] - box[1]) < 5 or np.linalg.norm(box[3] - box[0]) < 5:
                continue
            if box[8] >= 0.9:
                color = (0, 255, 0)
            elif box[8] >= 0.8:
                color = (255, 0, 0)
            cv2.line(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color, 2)
            cv2.line(img, (int(box[0]), int(box[1])), (int(box[4]), int(box[5])), color, 2)
            cv2.line(img, (int(box[6]), int(box[7])), (int(box[2]), int(box[3])), color, 2)
            cv2.line(img, (int(box[4]), int(box[5])), (int(box[6]), int(box[7])), color, 2)

            min_x = min(int(box[0] / scale), int(box[2] / scale), int(box[4] / scale), int(box[6] / scale))
            min_y = min(int(box[1] / scale), int(box[3] / scale), int(box[5] / scale), int(box[7] / scale))
            max_x = max(int(box[0] / scale), int(box[2] / scale), int(box[4] / scale), int(box[6] / scale))
            max_y = max(int(box[1] / scale), int(box[3] / scale), int(box[5] / scale), int(box[7] / scale))

            line = ','.join([str(min_x), str(min_y), str(max_x), str(max_y)]) + '\r\n'
            f.write(line)

    img = cv2.resize(img, None, None, fx=1.0 / scale, fy=1.0 / scale, interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(os.path.join("data/results", base_name), img)


In [17]:
def ctpn(sess, net, image_name):
    img = cv2.imread(image_name)
    plt.imshow(img)
    plt.show()
    img, scale = resize_im(img, scale=TextLineCfg.SCALE, max_scale=TextLineCfg.MAX_SCALE)
    plt.imshow(img)
    plt.show()
    print("scale",scale)
    scores, boxes = test_ctpn(sess, net, img)
    print("scores",scores)
    print("boxes",boxes)
    textdetector = TextDetector()
    boxes = textdetector.detect(boxes, scores[:, np.newaxis], img.shape[:2])
    draw_boxes(img, image_name, boxes, scale)

    print('{:d} object proposals'.format(boxes.shape[0]))


In [18]:
tf.reset_default_graph() 
if os.path.exists("data/results/"):
    shutil.rmtree("data/results/")
os.makedirs("data/results/")

cfg_from_file('ctpn/text.yml')

# init session
config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)
# load network
net = get_network("VGGnet_test")
# load model
print(('Loading network {:s}... '.format("VGGnet_test")), end=' ')
saver = tf.train.Saver()

try:
    ckpt = tf.train.get_checkpoint_state(cfg.TEST.checkpoints_path)
    print('Restoring from {}...'.format(ckpt.model_checkpoint_path), end=' ')
    saver.restore(sess, ckpt.model_checkpoint_path)
    print('done')
except:
    raise 'Check your pretrained {:s}'.format(ckpt.model_checkpoint_path)

im = 128 * np.ones((300, 300, 3), dtype=np.uint8)
for i in range(2):
    _, _ = test_ctpn(sess, net, im)

im_names = glob.glob(os.path.join(cfg.DATA_DIR, 'demo', '*.png')) + \
           glob.glob(os.path.join(cfg.DATA_DIR, 'demo', '*.jpg'))

Tensor("Placeholder:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("conv5_3/conv5_3:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_conv/3x3/rpn_conv/3x3:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("lstm_o/Reshape_2:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("lstm_o/Reshape_2:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_cls_score/Reshape_1:0", shape=(?, ?, ?, 20), dtype=float32)
Tensor("rpn_cls_prob:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("Reshape_2:0", shape=(?, ?, ?, 20), dtype=float32)
Tensor("rpn_bbox_pred/Reshape_1:0", shape=(?, ?, ?, 40), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)
anchors.shape (10, 4)
num_anchors 10
deltas Tensor("rois/strided_slice_7:0", shape=(?,), dtype=float32) Tensor("rois/strided_slice_8:0", shape=(?,), dtype=float32) Tensor("rois/strided_slice_9:0", shape=(?,), dtype=float32) Tensor("rois/strided_slice_10:0", shape=(?,), dtype=float32)
Loading network VGGnet_test...  Restoring from checkpoints/VGGnet_f

In [19]:
for im_name in net.layers:
    print(('{:s}'.format(im_name)))

data
im_info
conv1_1
conv1_2
pool1
conv2_1
conv2_2
pool2
conv3_1
conv3_2
conv3_3
pool3
conv4_1
conv4_2
conv4_3
pool4
conv5_1
conv5_2
conv5_3
rpn_conv/3x3
lstm_o
rpn_bbox_pred
rpn_cls_score
rpn_cls_score_reshape
rpn_cls_prob
rpn_cls_prob_reshape
rpn_rois
rpn_targets
rois


In [22]:
net.get_output('rois')[0]

<tf.Tensor 'rois/Reshape_4:0' shape=(?, 5) dtype=float32>

In [10]:
export_base_path = 'tf-exports'
if not os.path.exists(export_base_path):
    os.makedirs(export_base_path)

protobuf_path = os.path.join(export_base_path, 'v1.pbtxt')
checkpoint_path = os.path.join(export_base_path, 'v1.ckpt')

tf.train.write_graph(sess.graph_def, '.', protobuf_path)

saver = tf.train.Saver()
saver.save(sess, save_path = checkpoint_path)

'tf-exports/v1.ckpt'